# Clustering Crypto

In [48]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [49]:
# Load the crypto_data.csv dataset.
crypto_org_df = pd.read_csv("crypto_data.csv", index_col=0)
crypto_df = crypto_org_df.copy()
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [50]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df.IsTrading]
crypto_df

# Alternative
# crypto_df[crypto_df['IsTrading']==True]

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [51]:
# Keep all the cryptocurrencies that have a working algorithm.
na_algorrithms = crypto_df.isna().sum()["Algorithm"]

if na_algorrithms==0:
    print("No need to remove any cryptocurrency")
else:
    print(f"{na_algorrithms} cryptocurrencies to be removed")

No need to remove any cryptocurrency


In [52]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns="IsTrading", inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [53]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)

In [54]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"]>0]

In [55]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = pd.DataFrame(crypto_df["CoinName"])
crypto_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [56]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns="CoinName", inplace=True)
crypto_df.head()

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [57]:
# Use get_dummies() to create variables for text features.

# But first ensure data types are the correct ones:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532 entries, 42 to XBC
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Algorithm        532 non-null    object 
 1   ProofType        532 non-null    object 
 2   TotalCoinsMined  532 non-null    float64
 3   TotalCoinSupply  532 non-null    object 
dtypes: float64(1), object(3)
memory usage: 20.8+ KB


In [58]:
# It looks like 'TotalCoinSupply' can be casted to float64.
crypto_df["TotalCoinSupply"] = crypto_df["TotalCoinSupply"].astype('float64')

<ipython-input-58-e1a5bcc6b0a8>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [59]:
# Now pd.get_dummies(···) can be properly used.
crypto_df = pd.get_dummies(crypto_df)
crypto_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
# Standardize the data with StandardScaler().
sc = StandardScaler()

crypto_scaled = sc.fit_transform(crypto_df)
crypto_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [61]:
# Using PCA to reduce dimension to three principal components.

# Initialize PCA model
pca = PCA(n_components=3)

# Get three principal components for the crypto data.
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.33815188,  1.0250945 , -0.52368469],
       [-0.32148464,  1.02535273, -0.52409913],
       [ 2.31402847,  1.70777411, -0.55747814],
       ...,
       [ 0.32990133, -2.26633328,  0.39091663],
       [-0.16269292, -2.03969656,  0.39489685],
       [-0.29402674,  0.68961042, -0.23294101]])

In [62]:
# Create a DataFrame with the three principal components.
crypto_pca_df = pd.DataFrame(data=crypto_pca, columns=["PC1","PC2","PC3"])
crypto_pca_df.head()

,PC1,PC2,PC3
0,-0.338152,1.025095,-0.523685
1,-0.321485,1.025353,-0.524099
2,2.314028,1.707774,-0.557478
3,-0.139786,-1.263456,0.202624
4,-0.152021,-1.998660,0.327761


In [63]:
# Reestablish indexes:
crypto_pca_df.index = crypto_df.index

In [64]:
# Fetch the explained variance
pca.explained_variance_ratio_

array([0.02793139, 0.0213774 , 0.02045279])

In [65]:
print(f"Only {round(pca.explained_variance_ratio_.sum()*100,2)}% of variance is explained by PC1-3")

Only 6.98% of variance is explained by PC1-3


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [66]:
# Create an elbow curve to find the best value for K.

inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(crypto_pca_df)
   inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [67]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)
model.labels_

array([0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2,
       2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0,
       0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2,
       0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0,
       2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0,
       0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0,
       0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2,
       0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0,

In [68]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.

#  Add a new column, "Class" to the crypto_pca_df DataFrame that holds the predictions.
crypto_pca_df["class"] = model.labels_
crypto_pca_df.head()

# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_pca_df.join(crypto_org_df)[['CoinName',
                                  'Algorithm',
                                  'ProofType',
                                  'TotalCoinsMined',
                                  'TotalCoinSupply',
                                  'PC1', 'PC2', 'PC3',
                                   'class']]

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.338152,1.025095,-0.523685,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321485,1.025353,-0.524099,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.314028,1.707774,-0.557478,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.139786,-1.263456,0.202624,2
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.152021,-1.998660,0.327761,2
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000,-0.168502,-1.141928,-0.014836,2
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000,-0.389762,1.280092,-0.446548,0
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0,-0.151713,-2.169782,0.309508,2
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000,-0.150462,-1.998743,0.327738,2
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000,-0.162692,-2.039697,0.394897,2


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [69]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_pca_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [70]:
# Create a table with tradable cryptocurrencies.

tradeable_currencies = clustered_df[['CoinName',
                                  'Algorithm',
                                  'ProofType',
                                  'TotalCoinsMined',
                                  'TotalCoinSupply',
                                   'class']]
tradeable_currencies

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,2
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,2
...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000,0
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000,0
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610,2
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000,2


In [71]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {tradeable_currencies.shape[0]} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [72]:
tradeable_currencies.dtypes

CoinName            object
Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
class                int32
dtype: object

In [73]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

# Again 'TotalCoinSupply' can be casted to float64.
tradeable_currencies["TotalCoinSupply"] = tradeable_currencies["TotalCoinSupply"].astype('float64')

sc = StandardScaler()

tradeable_currencies_scaled = sc.fit_transform(tradeable_currencies[["TotalCoinsMined","TotalCoinSupply"]])
tradeable_currencies_scaled

<ipython-input-73-80544091d526>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([[-0.11710817, -0.1528703 ],
       [-0.09396955, -0.145009  ],
       [ 0.52494561,  4.48942416],
       ...,
       [-0.09561336, -0.13217937],
       [-0.11694817, -0.15255998],
       [-0.11710536, -0.15285552]])

,TotalCoinsMined,TotalCoinSupply
42,-0.117108,-0.152870
404,-0.093970,-0.145009
1337,0.524946,4.489424
BTC,-0.116715,-0.152560
ETH,-0.114747,-0.152870


In [77]:
tradeable_currencies_scaled_df = tradeable_currencies_scaled_df.join(tradeable_currencies[["CoinName","class"]])

In [78]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
tradeable_currencies_scaled_df = pd.DataFrame(data=tradeable_currencies_scaled,
                                              columns=["TotalCoinsMined","TotalCoinSupply"],
                                              index=tradeable_currencies.index)
tradeable_currencies_scaled_df.head()

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = tradeable_currencies_scaled_df.join(tradeable_currencies[["CoinName","class"]])


plot_df.head(10)

,TotalCoinsMined,TotalCoinSupply,CoinName,class
42,-0.117108,-0.152870,42 Coin,0
404,-0.093970,-0.145009,404Coin,0
1337,0.524946,4.489424,EliteCoin,0
BTC,-0.116715,-0.152560,Bitcoin,2
ETH,-0.114747,-0.152870,Ethereum,2
LTC,-0.115726,-0.151629,Litecoin,2
DASH,-0.116910,-0.152545,Dash,0
XMR,-0.116731,-0.152870,Monero,2
ETC,-0.114622,-0.149767,Ethereum Classic,2
ZEC,-0.116946,-0.152560,ZCash,2


In [79]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["class"],
    by="class",
)

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,class)